In [1]:
import unstructured_client 
from unstructured_client.models import operations , shared 

In [ ]:
import dotenv 
import os 
dotenv.load_dotenv()
UNSTRUCTURED_API_KEY=os.environ["UNSTRUCTURED_API_KEY"]

uio_client = unstructured_client.UnstructuredClient(
          api_key_auth=UNSTRUCTURED_API_KEY
         )

In [4]:
def get_request( file : bytes , filename : str ,  **kwargs):
        # Default parameters

    
        defaults = {
            "strategy": "hi_res",
            "chunking_strategy": "by_title",
            "max_characters": 3000, ### for testing 
            "new_after_n_chars": 2400,
            "unique_element_ids": True,
            "extract_image_block_types": ["Image", "Table"],
            "pdf_infer_table_structure": True,
            "include_page_breaks": False,
            "split_pdf_page": True,
            "split_pdf_concurrency_level": 15,
            "include_orig_elements": True
        }
        
        # Override defaults with any provided kwargs
        params = {**defaults, **kwargs}
        
        request = operations.PartitionRequest(
            partition_parameters=shared.PartitionParameters(
                files=shared.Files(
                    content=file,
                    file_name=filename,
                ),
                **params
            )
        )
        
        return request 

In [3]:
pdf_path = "ocs.pdf" 
with open(pdf_path , "rb") as file : 
    file_data  = file.read()

In [5]:
request = get_request(file=file_data , filename="ocs")
response = await uio_client.general.partition_async(request=request)

INFO: HTTP Request: GET https://api.unstructuredapp.io/general/docs "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 200 OK"


In [8]:
chunk = response.elements[0]

In [10]:
orig_elements = chunk["metadata"]["orig_elements"]  

In [11]:
orig_elements

'eJztXGlz20iS/SsIf+oOSCrcx3TsB/AEb/AUxZmNDhwFAiQIgDgIUhP73zcLIClZotvT3vbGHnSEbFWhMivr5cuXRSngv//zCw7wDofZ777z5W/UF54XbY61lUfRYcRHgVHtR4XjhUfXVExT4RWFk8QvD9SXHc5Mx8xMsPnnFzuKEscPzQyn5TgwT1Ge/e5hf+1lMMPxvAo25+nCdzIPZllJlGA2jvwwI3Z//zsr8E/iA8Wq6hPz7w/UZcwJypNYjkVGfGJvTFQWMPMlPaUZ3pGTGP4RB9PYtPGX/4AHDs6wnflR+LsdmGn6e5xEFixjniReFmRY4PoBzk4xLm2NATmkvzPX+HfLTLEkkGmkbpCgaePpdtWdrLWaNtZgBP9qDcQVtYamrdu1td2ujdd6rbC79fG616iNhw0t7Tdq22mzUOb6JHJ1jzFhwbpbF8JOvZN2joXda9hxv6Edh40xo59s0ZitT0ZdeB02+pXv8Rb8FeBnPWxM1qfOpPAGm87pv/bVR4WmTeqadqotmg1t2tE0r1nTjs0aUsaaXpDzzeGcTfKlvf1p1rVi3IKzdurjqE/WZhOYXoOPotPUinmrAqelauMO4DKe1CYdbzBvtptsy6udusdWo1fbms1Oh+kdi8liyrjadsB1T+v1ttfy7Jf2JAq6vh31phEznL3wo8ZWGjOT1mTrdKbzeLZoLZ4XrLd63g23q+dVaLaDvc1PWCd0BLzcN7zOc9sT/K4f92ZB9/k5EP3Vbt/bxL3nXSz58T7pvWb9Zy6TfTGH9BwHy/ZR2XRPWX/GDpfPrLpZcbnuC/QsEHM3knByVA4Go7oWTxuhVOEDmBTNCz7Vkb+JT5PgU4dFjZpmV/iMu3y59ths1qbNY83r1uZjx7MnA38QNTutZm8SWXqtPh28zg/a4iXsT+rey3Y4agbKpMRoB4jFvc2cG76uxdF22JzMV/q0GfTm7GSxCJyX50W8WbUXuxXnxWY4ZJzlisd6IK2

In [26]:


import json 
from pathlib import Path 
import base64
import zlib

def elements_from_base64_gzipped_json(b64_encoded_elements: str):
    """Manually decode base64 gzipped JSON elements"""
    # Base64 decode
    decoded_bytes = base64.b64decode(b64_encoded_elements)
    # Decompress gzip
    decompressed_bytes = zlib.decompress(decoded_bytes)
    # Decode to string and parse JSON
    elements_json = decompressed_bytes.decode('utf-8')
    element_dicts = json.loads(elements_json)
    return element_dicts


In [27]:
atomic_elements = elements_from_base64_gzipped_json(orig_elements) 

In [28]:
atomic_elements

[{'element_id': '335c21c8-5d05-409c-8234-fa8aa8388265',
  'metadata': {'coordinates': {'layout_height': 2339,
    'layout_width': 1656,
    'points': [[143.5, 199.0],
     [143.5, 248.5],
     [1505.1, 248.5],
     [1505.1, 199.0]],
    'system': 'PixelSpace'},
   'detection_class_prob': 0.63747,
   'filetype': 'PPM',
   'image_base64': '/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/2wBDAQkJCQwLDBgNDRgyIRwhMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjL/wAARCAAyBVEDASIAAhEBAxEB/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/8QAHwEAAwEBAQEBAQEBAQAAAAAAAAECAwQFBgcICQoL/8QAtREAAgECBAQDBAcFBAQAAQJ3AAECAxEEBSExBhJBUQdhcRMiMoEIFEKRobHBCSMzUvAVYnLRChYkNOEl8RcYGRom